<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/SEAL_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/Continual-Intelligence/SEAL/tree/main

In [ ]:
!git clone https://github.com/Continual-Intelligence/SEAL.git
%cd SEAL
!pip install -r requirements.txt -q
!pip install colab-env -q
import colab_env

In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
!sh /content/SEAL/few-shot/launch.sh

In [5]:
DATA_DIR='/content/SEAL/few-shot/data'
!python /content/SEAL/few-shot/self-edit.py  \
    --experiment_name=training_set_iteration_1 \
    --challenge_file={DATA_DIR}/arc-agi_training_challenges_filtered_1B_training_set.json \
    --solution_file={DATA_DIR}/arc-agi_training_solutions_filtered_1B_training_set.json \
    --model_name=meta-llama/Llama-3.2-1B-Instruct \
    --n_tasks=12 \
    --n_self_edits_per_task=15

Streaming output truncated to the last 5000 lines.
{'loss': 0.008, 'grad_norm': 0.017266247421503067, 'learning_rate': 9.410911550880474e-06, 'epoch': 2.86}
{'loss': 0.0091, 'grad_norm': 0.017133334651589394, 'learning_rate': 6.541645633054649e-06, 'epoch': 2.89}
{'loss': 0.011, 'grad_norm': 0.01525526400655508, 'learning_rate': 4.189949386787462e-06, 'epoch': 2.92}
{'loss': 0.0085, 'grad_norm': 0.01685592345893383, 'learning_rate': 2.3582894166930268e-06, 'epoch': 2.94}
{'loss': 0.0085, 'grad_norm': 0.019107569009065628, 'learning_rate': 1.0485868811441758e-06, 'epoch': 2.97}
{'loss': 0.0092, 'grad_norm': 0.01672017015516758, 'learning_rate': 2.6221547724253336e-07, 'epoch': 3.0}
{'train_runtime': 14.3203, 'train_samples_per_second': 15.084, 'train_steps_per_second': 7.542, 'train_loss': 0.2811508280682136, 'epoch': 3.0}
100% 108/108 [00:14<00:00,  7.54it/s]
Training complete.
Model saved to loras/self-edit/training_set_iteration_1/8be77c9e/7
Training on 250 examples for 3 epochs, lr:

In [ ]:
!python /content/SEAL/few-shot/ttt.py

2025-07-03 10:10:26.564160: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 10:10:26.581375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751537426.602583   52488 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751537426.609044   52488 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 10:10:26.630323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
DATA_DIR='/content/SEAL/few-shot/data'
TTI_DIR='/content/SEAL/loras/self-edit/training_set_iteration_1'
LORA_DIR='/content/SEAL/loras/self-edit/training_set_iteration_1/8d5021e8/14'

!python /content/SEAL/few-shot/eval-self-edits.py  \
    --experiment_folder={TTI_DIR} \
    --pretrained_checkpoint=meta-llama/Llama-3.2-1B-Instruct \
    --lora_checkpoints_folder={LORA_DIR} \
    --temperature=0 \
    --n_sample=1 \
    --data_file=/content/SEAL/few-shot/data/arc-agi_training_challenges_filtered_1B_training_set.json \
    --solution_file=/content/SEAL/few-shot/data/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --max_lora_rank=128 \
    --include_n=1 \
    --new_format \
    --num_examples=11 \
    --n_self_edits=15

In [13]:
!python /content/SEAL/few-shot/BC-self-edit.py \
    --configs_and_indices=/content/SEAL/loras/self-edit/training_set_iteration_1/final_configs_and_indices.json \
    --results=/content/SEAL/few-shot/final_results.json \
    --model_name=meta-llama/Llama-3.2-1B-Instruct \
    --lora_rank=16 \
    --lora_alpha=16 \
    --num_train_epochs=8 \
    --per_device_train_batch_size=5 \
    --gradient_accumulation_steps=1 \
    --learning_rate=5e-5

2025-07-03 10:05:44.086943: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 10:05:44.104948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751537144.127281   50546 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751537144.133954   50546 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 10:05:44.155699: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
!python /content/SEAL/few-shot/self-edit.py  \
    --experiment_name=eval_RL_iteration_1_8_epoch \
    --challenge_file={DATA_DIR}/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file={DATA_DIR}/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --model_name={LORA_DIR}/self-edit/training_set_iteration_1/RL_trained_model_iteration_1_8_epoch \
    --n_tasks=10 \
    --n_self_edits_per_task=5

2025-07-03 06:55:06.033917: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 06:55:06.050785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751525706.072214    5328 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751525706.078762    5328 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 06:55:06.099725: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [7]:
!python /content/SEAL/few-shot/eval-self-edits.py \
    --experiment_folder=${TTI_DIR}/eval_set_RL_iteration_1_8_epoch \
    --pretrained_checkpoint=${LORA_DIR}/self-edit/training_set_iteration_1/RL_trained_model_iteration_1_8_epoch \
    --lora_checkpoints_folder=${LORA_DIR}/self-edit/eval_RL_iteration_1_8_epoch \
    --temperature=0 \
    --n_sample=1 \
    --data_file=${DATA_DIR}/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file=${DATA_DIR}/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --max_lora_rank=128 \
    --include_n=1 \
    --new_format \
    --num_examples=9 \
    --n_self_edits=5

2025-07-03 06:55:19.953865: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 06:55:19.972063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751525719.993849    5454 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751525720.000600    5454 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 06:55:20.022504: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [8]:
!python /content/SEAL/few-shot/eval-self-edits-baseline.py \
    --experiment_folder=${TTI_DIR}/eval_base_model \
    --pretrained_checkpoint=meta-llama/Llama-3.2-1B-Instruct \
    --lora_checkpoints_folder=${LORA_DIR}/self-edit/eval_RL_iteration_1_8_epoch \
    --temperature=0 \
    --n_sample=1 \
    --data_file=${DATA_DIR}/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file=${DATA_DIR}/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --max_lora_rank=128 \
    --include_n=1 \
    --new_format \
    --num_examples=9

2025-07-03 06:55:33.325673: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 06:55:33.342930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751525733.364423    5577 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751525733.370987    5577 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 06:55:33.392017: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [9]:
!python /content/SEAL/few-shot/self-edit.py \
    --experiment_name=eval_RL_iteration_1 \
    --challenge_file=${DATA_DIR}/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file=${DATA_DIR}/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --model_name=${LORA_DIR}/self-edit/training_set_iteration_1/RL_trained_model_iteration_1 \
    --n_tasks=10 \
    --n_self_edits_per_task=5

2025-07-03 06:55:47.204344: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 06:55:47.221330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751525747.242423    5697 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751525747.248884    5697 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 06:55:47.269756: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr